In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

# eda_final_project

## Duomenys

Duomenų šaltinis https://www.registrucentras.lt/p/1094. 

In [3]:
import pandas as pd
import numpy as np

<IPython.core.display.Javascript object>

In [4]:
# https://stackoverflow.com/questions/21137150/format-suppress-scientific-notation-from-pandas-aggregation-results
pd.set_option("display.float_format", lambda x: "%.2f" % x)

<IPython.core.display.Javascript object>

In [5]:
## got an error while downloading online csv data. OSX
## quick fix
## https://stackoverflow.com/questions/50236117/scraping-ssl-certificate-verify-failed-error-for-http-en-wikipedia-org

import ssl

ssl._create_default_https_context = ssl._create_unverified_context

<IPython.core.display.Javascript object>

# Pirma dalis

### 2 
Parsisiųsti 2020 ir 2021 įmonių “Juridinių asmenų pateikti finansinės atskaitomybės dokumentai – balanso ataskaitos” ir “Juridinių asmenų pateikti finansinės atskaitomybės dokumentai – pelno (nuostolių) ataskaitos” duomenis.

In [ ]:
balance_20 = pd.read_csv(
    "https://www.registrucentras.lt/aduomenys/?byla=JAR_FA_RODIKLIAI_BLNS_2020.csv"
)
balance_21 = pd.read_csv(
    "https://www.registrucentras.lt/aduomenys/?byla=JAR_FA_RODIKLIAI_BLNS_2021.csv"
)
pl_20 = pd.read_csv(
    "https://www.registrucentras.lt/aduomenys/?byla=JAR_FA_RODIKLIAI_PLNA_2020.csv"
)
pl_21 = pd.read_csv(
    "https://www.registrucentras.lt/aduomenys/?byla=JAR_FA_RODIKLIAI_PLNA_2021.csv"
)

In [ ]:
balance_20.head()

In [ ]:
pl_20.head()

### 3. Susipažinkite su duomenimis

In [ ]:
balance_20.shape, balance_21.shape, pl_20.shape, pl_21.shape

In [ ]:
balance_20.dtypes

In [ ]:
balance_21.dtypes == balance_20.dtypes

In [ ]:
# Count null values
print(balance_20.isnull().sum().sum())
print(balance_21.isnull().sum().sum())
print(pl_20.isnull().sum().sum())
print(pl_21.isnull().sum().sum())

In [ ]:
# Count null values
balance_21.isnull().sum(axis=0)



In [ ]:
pl_21.isnull().sum(axis=0)


In [ ]:
print(balance_20[balance_20.duplicated()].shape)
print(balance_21[balance_21.duplicated()].shape)
print(pl_20[pl_20.duplicated()].shape)
print(pl_21[pl_21.duplicated()].shape)

Daugiausiai nežinomų verčių turi pl_21 lentelė ir *pelnas_pries_apmokestinima* stulpelis

# Antra dalis

In [ ]:
# 4. Raskite bendroves su Thermo Fisher pavadinimu
pl_20[pl_20['obj_pav'].str.contains("Thermo Fisher")].obj_pav.values

In [ ]:
# 5. Apsimeskime, kad atributas “form_pav” neegzistuoja. Iš likusių laukelių gaukite visas UAB ir MB bendroves. Palyginkite rekonstrukcijos tikslumą su “form_pav” vertėmis - kiek procentų verčių pavyko atkurti skirtingose duomenyse?
pl_21['extracted_form_pav'] = pl_21.obj_pav.str.split(" \"").str.get(0)
pl_20['extracted_form_pav'] = pl_20.obj_pav.str.split(" \"").str.get(0)
balance_20['extracted_form_pav'] = balance_20.obj_pav.str.split(" \"").str.get(0)
balance_21['extracted_form_pav'] = pl_21.obj_pav.str.split(" \"").str.get(0)

print(pl_21.extracted_form_pav.value_counts().head(40))

new_mapping = {'UAB': 'Uždaroji akcinė bendrovė', 'MB': 'Mažoji bendrija', 'Uždaroji akcinė bendrovė': 'Uždaroji akcinė bendrovė', 'Mažoji bendrija': 'Mažoji bendrija', 'UŽDAROJI AKCINĖ BENDROVĖ':'Uždaroji akcinė bendrovė'}
pl_21.extracted_form_pav = pl_21.extracted_form_pav.map(new_mapping)
pl_20.extracted_form_pav = pl_20.extracted_form_pav.map(new_mapping)
balance_20.extracted_form_pav = balance_20.extracted_form_pav.map(new_mapping)
balance_21.extracted_form_pav = balance_21.extracted_form_pav.map(new_mapping)

In [ ]:
orig_filt = pl_20[pl_20['form_pav'].isin(["Uždaroji akcinė bendrovė", "Mažoji bendrija"])]['form_pav']
extracted_filt = pl_20['extracted_form_pav'].dropna()
print(f"Share: {100*extracted_filt.shape[0]/orig_filt.shape[0]}")

orig_filt = pl_21[pl_21['form_pav'].isin(["Uždaroji akcinė bendrovė", "Mažoji bendrija"])]['form_pav']
extracted_filt = pl_21['extracted_form_pav'].dropna()
print(f"Share: {100*extracted_filt.shape[0]/orig_filt.shape[0]}")

orig_filt = balance_20[balance_20['form_pav'].isin(["Uždaroji akcinė bendrovė", "Mažoji bendrija"])]['form_pav']
extracted_filt = balance_20['extracted_form_pav'].dropna()
print(f"Share: {100*extracted_filt.shape[0]/orig_filt.shape[0]}")

orig_filt = balance_21[balance_21['form_pav'].isin(["Uždaroji akcinė bendrovė", "Mažoji bendrija"])]['form_pav']
extracted_filt = balance_21['extracted_form_pav'].dropna()
print(f"Share: {100*extracted_filt.shape[0]/orig_filt.shape[0]}")


### 6 
Apskaičiuoti vidutinias reikšmes “pelnas_pries_apmokestinima” ir “nuosavas_kapitalas” laukų kiekvienai juridinio asmens formai (Uždaroji akcinė bendrovė, Mažoji bendrija, etc.) už visą laikotarpį (2020 ir 2021 metai kartu). 

In [ ]:
balance_20.form_pav.value_counts(dropna=False)

In [ ]:
# nuosavas_kapitalas
nuosavas_kapitalas = balance_20[["obj_kodas", "nuosavas_kapitalas"]]

In [ ]:
nuosavas_kapitalas.shape

In [ ]:
# form_pav,  pelnas_pries_apmokestinima
pelnas_pries_apmokestinima = pl_20[
    ["obj_kodas", "form_pav", "pelnas_pries_apmokestinima"]
]

In [ ]:
pelnas_pries_apmokestinima.shape

In [ ]:
balance_20_set = set(balance_20.obj_kodas)
pl_20_set = set(pl_20.obj_kodas)

len(pl_20_set.difference(balance_20_set)), len(balance_20_set.difference(pl_20_set))

In [ ]:
# we take intersection and ignore companies that do not have profit loss statement.
pelnas_pries_apmokestinima.merge(nuosavas_kapitalas, on="obj_kodas", how="inner").shape

In [ ]:
# we see there are duplicates !!!!

In [ ]:
balance_20.obj_kodas.value_counts().iloc[:3]

In [ ]:
balance_20[balance_20.laikotarpis_nuo == "2019-01-01"].obj_kodas.value_counts()

In [ ]:
# we filter out duplicates(?) by choosing laikotarpis_nuo == "2019-01-01". This requires inside knowledge about balance sheets.
nuosavas_kapitalas = balance_20[balance_20.laikotarpis_nuo == "2019-01-01"][
    ["obj_kodas", "nuosavas_kapitalas"]
]

In [ ]:
pelnas_pries_apmokestinima = pl_20[pl_20.laikotarpis_nuo == "2019-01-01"][
    ["obj_kodas", "form_pav", "pelnas_pries_apmokestinima"]
]

In [ ]:
pelnas_pries_apmokestinima.obj_kodas.value_counts()  ## no duplicates.

In [ ]:
nuosavas_kapitalas.shape, pelnas_pries_apmokestinima.shape

In [ ]:
# we drop nan and use inner merge
merged = nuosavas_kapitalas.merge(
    pelnas_pries_apmokestinima, on="obj_kodas", how="inner"
)

In [ ]:
year_20_result = merged.groupby('form_pav').agg({'obj_kodas':'count', 
                         'nuosavas_kapitalas':'mean', 
                         'pelnas_pries_apmokestinima':'mean', }).apply(round).rename(columns=dict(obj_kodas='count'))


In [ ]:
year_20_result

In [ ]:
## year 2021

In [ ]:
nuosavas_kapitalas = balance_21[balance_21.laikotarpis_nuo == "2020-01-01"][
    ["obj_kodas", "nuosavas_kapitalas"]
]
pelnas_pries_apmokestinima = pl_21[pl_21.laikotarpis_nuo == "2020-01-01"][
    ["obj_kodas", "form_pav", "pelnas_pries_apmokestinima"]
]
## check for duplicates.
nuosavas_kapitalas.shape, pelnas_pries_apmokestinima.shape, len(
    set(nuosavas_kapitalas.obj_kodas)
), len(set(pelnas_pries_apmokestinima.obj_kodas))
#  No duplicates!!

In [ ]:
merged = nuosavas_kapitalas.merge(
    pelnas_pries_apmokestinima, on="obj_kodas", how="inner"
)
year_21_result = merged.groupby('form_pav').agg({'obj_kodas':'count', 
                         'nuosavas_kapitalas':'mean', 
                         'pelnas_pries_apmokestinima':'mean', }).apply(round).rename(columns=dict(obj_kodas='count'))


In [ ]:
year_21_result

In [ ]:
merged_result = pd.concat(
    {"2020": year_20_result, "2021": year_21_result}, axis=1
).fillna(0)

In [ ]:
merged_result

# 3
Atrinkti UAB ir MB įmones, kurių “nuosavas_kapitalas” buvo didesnis negu nuosavo kapitalo mediana už visą laitotarpį (2020 ir 2021 metai kartu). 

In [ ]:
uab_condition_21 = balance_21.form_pav == "Uždaroji akcinė bendrovė"
uab_condition_20 = balance_20.form_pav == "Uždaroji akcinė bendrovė"
laikotarpis_condition_21 = balance_21.laikotarpis_nuo == "2020-01-01"
laikotarpis_condition_20 = balance_20.laikotarpis_nuo == "2019-01-01"

capital_median_21 = balance_21[laikotarpis_condition_21 & uab_condition_21][
    ["obj_kodas", "nuosavas_kapitalas"]
].nuosavas_kapitalas.median()

capital_median_20 = balance_20[laikotarpis_condition_20 & uab_condition_20][
    ["obj_kodas", "nuosavas_kapitalas"]
].nuosavas_kapitalas.median()

In [ ]:
capital_median_20, capital_median_21

In [ ]:
balance_20[
    laikotarpis_condition_20
    & uab_condition_20
    & (balance_20.nuosavas_kapitalas > capital_median_20)
].sort_values(by="nuosavas_kapitalas").tail(5)

In [ ]:
balance_21[
    laikotarpis_condition_21
    & uab_condition_21
    & (balance_21.nuosavas_kapitalas > capital_median_21)
].sort_values(by="nuosavas_kapitalas").tail(5)

In [ ]:
# Įvertinkite kurios įmonės išsiskiria ypač didele “nuosavas_kapitalas” verte (patenka į 99th percentilę)
balance_21.nuosavas_kapitalas.describe(percentiles=[0.25, 0.5, 0.75, 0.99])
balance_21[balance_21.nuosavas_kapitalas > 6364687.99][['obj_pav', 'nuosavas_kapitalas']].sort_values("nuosavas_kapitalas", ascending=False)[:10]

### 8. 
Įvertinkite kurios įmonės išsiskiria ypač didele “nuosavas_kapitalas” verte (patenka į 99th percentilę). Patikrinkite įmonių patenkančių į TOP-10 vertes rekvizitai.vz.lt svetainėje. Ranka įrašomi duomenys sukuria klaidos galimybę. Ar yra bendrovė, kurios “nuosavas_kapitalas” jums kelia įtarimų?

In [ ]:
# Įvertinkite kurios įmonės išsiskiria ypač didele “nuosavas_kapitalas” verte (patenka į 99th percentilę)
balance_20.nuosavas_kapitalas.describe(percentiles=[0.25, 0.5, 0.75, 0.99])
balance_20[balance_20.nuosavas_kapitalas > balance_20.nuosavas_kapitalas.describe(percentiles=[0.25, 0.5, 0.75, 0.99])['99%']][['obj_pav', 'nuosavas_kapitalas']].sort_values("nuosavas_kapitalas", ascending=False)[:10]

### 9.
Vizualiai atvaizduoti juridinių asmenų formos pasiskirstymą per visą laikotarpį (2020 ir 2021 metais kartu). 

In [ ]:
b20 = (
    balance_20.groupby("form_pav", as_index=False)
    .form_kodas.count()
    .rename(columns={"form_kodas": "2020_count"})
)

In [ ]:
b21 = (
    balance_21.groupby("form_pav", as_index=False)
    .form_kodas.count()
    .rename(columns={"form_kodas": "2021_count"})
)

In [ ]:
data = b20.merge(b21, on="form_pav", how="outer").fillna(0)

In [ ]:
data

In [ ]:
# With matplotlib

import matplotlib.pyplot as plt

X_axis = np.arange(data.shape[0])
  
plt.bar(X_axis - 0.2, data['2020_count'], 0.4, label = '2020_count')
plt.bar(X_axis + 0.2, data['2021_count'], 0.4, label = '2021_count')

plt.grid()
plt.yscale('log')
plt.xticks(X_axis, data['form_pav'], rotation = 90)
plt.ylabel("Vienetų skaičius")
plt.title("Juridinių asmenų skaičius 2020 ir 2021 metais")
plt.legend()
plt.show()

In [ ]:
# With pandas
data.set_index('form_pav').plot(kind='bar', ylabel='Vienetų skaičius', logy=True, grid=True, xlabel=None, title='Juridinių asmenų skaičius 2020 ir 2021 metais')

In [ ]:
# seaborn
import seaborn as sns
data_melted =  data.melt(id_vars='form_pav')

ax = sns.barplot(data=data_melted, x='form_pav', y='value', hue='variable', orient='v')
ax.set_yscale("log")
ax.set_ylabel("Vienetų skaičius")
ax.set_title("Juridinių asmenų skaičius 2020 ir 2021 metais")
plt.grid()
plt.xticks(rotation=90)

In [ ]:
b20.merge(b21, on="form_pav", how="outer")

In [ ]:
import seaborn as sns

In [ ]:
sns.barplot(data=b20, x="form_pav", y="2020_count")

### 10. Kurį mėnesį buvo įregistruota daugiausiai juridinių asmenų 2020 ir 2021 metais?


In [ ]:
balance_20['reg_menesis'] = balance_20['reg_date'].str.split('-').str.get(1)
balance_21['reg_menesis'] = balance_21['reg_date'].str.split('-').str.get(1)


year_2020 = balance_20.groupby(by='reg_menesis').count()['obj_kodas']
year_2021 = balance_21.groupby(by='reg_menesis').count()['obj_kodas']

In [ ]:
pd.concat([year_2020, year_2021], axis=1).plot(kind='bar')

### 11. Vizualiai atvaizduoti juridinių asmenų statuso pasiskirstymą kiekvienais metais (2020 ir 2021 metais kartu).

Check exercice 9 with stat_statusas feature.

### 12  Kiekvienai įmonei apskaičiuoti tendenciją. Jei “grynasis_pelnas” + “nuosavas_kapitalas” 2021 metais buvo didesnis už 2020 metais deklaruotą, tai tendencija “teigiama”, jei ne - “neigiama”. Atvaizduoti tendencijos duomenis stulpelinėje diagramoje.

In [ ]:
tend_20 = pl_20[["obj_kodas", "form_pav", "grynasis_pelnas"]].merge(
    balance_20[["obj_kodas", "nuosavas_kapitalas"]]
)

In [ ]:
tend_20["value_20"] = tend_20["grynasis_pelnas"] + tend_20["nuosavas_kapitalas"]
tend_20 = tend_20[["obj_kodas", "form_pav", "value_20"]]

In [ ]:
tend_21 = pl_21[["obj_kodas", "form_pav", "grynasis_pelnas"]].merge(
    balance_21[["obj_kodas", "nuosavas_kapitalas"]]
)
tend_21["value_21"] = tend_21["grynasis_pelnas"] + tend_21["nuosavas_kapitalas"]
tend_21 = tend_21[["obj_kodas", "value_21"]]

In [ ]:
tend = tend_20.merge(tend_21, on="obj_kodas")

In [ ]:
tend["tendencija"] = "neigiama"

In [ ]:
tend.loc[tend.value_21 > tend.value_20, "tendencija"] = "teigiama"

In [ ]:
tend.tendencija.value_counts()

# 13
Surasti UAB ir MB įmones, kurios 2020 vykdė veiklą, o 2021 - bankrutavo. Bankrutavusių įmonių statusai: “Bankrutavęs”, “Bankrutuojantis”, “Išregistruotas”, “Inicijuojamas likvidavimas”, “Likviduojamas”, “Likviduojamas dėl bankroto”. Toliau veiklą vykdanti įmonė pažymėta "Teisinis stat neįregistruotas" statusu. Kokia bankrutavusių įmonių “nuosavas_kapitalas”suma 2021 metais? Kokia bankrutavusių įmonių “grynasis_pelnas” suma 2021 metais? Palyginti šias sumas su veikiančių įmonių atitinkamomis sumomis.

In [ ]:
# dirbame tik su MB ir UAB
## veikianti imone = 'Teisinis stat neįregistruotas'
## Bankrutavusi imone = 'Bankrutavęs', 'Bankrutuojantis', 'Išregistruotas', 'Likviduojamas', 'Likviduojamas dėl bankroto', 'Inicijuojamas likvidavimas'

In [ ]:
def get_veikianti_bankrutavusi(x):
    veikianti = "Teisinis stat neįregistruotas"
    bankrutavusi = [
        "Bankrutavęs",
        "Bankrutuojantis",
        "Išregistruotas",
        "Likviduojamas",
        "Likviduojamas dėl bankroto",
        "Inicijuojamas likvidavimas",
    ]
    if x == veikianti:
        return "veikianti"
    elif x in bankrutavusi:
        return "bankrutavusi"
    return np.nan


balance_20["veikianti_bankrutavusi"] = balance_20.stat_pav.map(
    get_veikianti_bankrutavusi
)
balance_21["veikianti_bankrutavusi"] = balance_21.stat_pav.map(
    get_veikianti_bankrutavusi
)

In [ ]:
bal_cols = ["obj_kodas", "form_pav", "veikianti_bankrutavusi", "nuosavas_kapitalas"]
pl_cols = ["obj_kodas", "grynasis_pelnas"]
merge_on = "obj_kodas"
mb_uab = ["Mažoji bendrija", "Uždaroji akcinė bendrovė"]
uab_mb_codition_20 = balance_20.form_pav.isin(mb_uab)
uab_mb_codition_21 = balance_21.form_pav.isin(mb_uab)

merged_20 = balance_20[uab_mb_codition_20][bal_cols].merge(pl_20[pl_cols], on=merge_on)
merged_21 = balance_21[uab_mb_codition_21][bal_cols].merge(pl_21[pl_cols], on=merge_on)

In [ ]:
merged = merged_20.merge(merged_21, on=merge_on, suffixes=["_20", "_21"], how="left")

In [ ]:
merged.veikianti_bankrutavusi_20.value_counts(), merged.veikianti_bankrutavusi_21.value_counts(),

In [ ]:
veikianti_bankrutavusi_codition = merged.apply(
    lambda x: (x["veikianti_bankrutavusi_20"] == "veikianti")
    & (x["veikianti_bankrutavusi_21"] == "bankrutavusi"),
    axis=1,
)

In [ ]:
merged[
    merged.veikianti_bankrutavusi_21 == "bankrutavusi"
].veikianti_bankrutavusi_20.value_counts()

In [ ]:
## Nera bankrutavusiu imoniu?!

# 14
Sukurti naują stulpelį “turtas”, kuris skaičiuojamas sudedant “trumpalaikis_turtas” ir “ilgalaikis_turtas”.

In [ ]:
balance_20["turtas"] = (
    balance_20["trumpalaikis_turtas"] + balance_20["ilgalaikis_turtas"]
)

balance_21["turtas"] = (
    balance_21["trumpalaikis_turtas"] + balance_21["ilgalaikis_turtas"]
)

Suskirstyti įmones į 5 grupes pagal lauką “turtas”. Palyginti kaip šios grupės keičiasi 2020 ir 2021 metais. Palyginimą atvaizduoti grafiškai.

In [ ]:
balance_20.turtas.describe(percentiles=[0.2, 0.4, 0.6, 0.8])

In [ ]:
balance_20["turtas_category"] = pd.cut(
    balance_20.turtas,
    bins=[0, 3000, 20000, 35000, 70000, 3_000_000_000],
    labels=["iki 3k", "3k-20k", "20k-35k", "35k-70k", "70k+"],
)

In [ ]:
balance_21["turtas_category"] = pd.cut(
    balance_21.turtas,
    bins=[0, 3000, 20000, 35000, 70000, 3_000_000_000],
    labels=["iki 3k", "3k-20k", "20k-35k", "35k-70k", "70k+"],
)

In [ ]:
b20 = (
    balance_20[["obj_kodas", "turtas_category"]]
    .copy()
    .rename(columns={"turtas_category": "turtas_category_2020"})
)

In [ ]:
b21 = (
    balance_21[["obj_kodas", "turtas_category"]]
    .copy()
    .rename(columns={"turtas_category": "turtas_category_2021"})
)

In [ ]:
turtas = b20.merge(b21, on="obj_kodas")
turtas.head()

In [ ]:
## palyginimas -> count'as 
## judejimas is vienos kategorijos i kita 

In [ ]:
s_2021

In [ ]:
pd.concat([s_2020, s_2021],axis=1)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def autopct_format(values):
        def my_format(pct):
            total = sum(values)
            val = int(round(pct*total/100.0))
            return '{:.1f}%\n({v:d})'.format(pct, v=val)
        return my_format

s_2020 = turtas['turtas_category_2020'].value_counts()
s_2021= turtas['turtas_category_2021'].value_counts()
s = pd.concat([s_2020, s_2021],axis=1)

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 10))

ax[0].pie(s['turtas_category_2020'],labels = s.index, autopct=autopct_format(s['turtas_category_2020']))
ax[0].set_title("2020 metais")
ax[1].pie(s['turtas_category_2021'],labels = s.index, autopct=autopct_format(s['turtas_category_2021']))
ax[1].set_title("2021 metais")
plt.suptitle("Įregistruotų įmonių pasiskirstymas pagal sukauptą turtą", y=0.80)
plt.tight_layout()
plt.show()



In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 10))

s.plot(ax=ax[0], y='turtas_category_2020', kind='pie',
         autopct=autopct_format(s['turtas_category_2020']), startangle=270, title="2020 metais", legend=None)
ax[0].axis('off')

s.plot(ax=ax[1], y='turtas_category_2020', kind='pie',
         autopct=autopct_format(s['turtas_category_2021']), startangle=270, legend=None, title="2020 metais")
ax[1].axis('off')

plt.suptitle("Įregistruotų įmonių pasiskirstymas pagal sukauptą turtą", y=0.80)
plt.tight_layout()
plt.show()


# 15
Surasti top100 pelningiausias įmones pagal “grynasis pelnas” ir top100 įmones kurių “nuosavas kapitalas” didžiausias. Ar yra sutampančių įmonių šiuose sąrašuose? Skaičiuojant topus imti viso laikotarpio įmones.

In [ ]:
mb_uab = ["Mažoji bendrija", "Uždaroji akcinė bendrovė"]
uab_mb_codition_20 = balance_20.form_pav.isin(mb_uab)
uab_mb_codition_21 = balance_21.form_pav.isin(mb_uab)

laikotarpis_condition_20 = balance_20.laikotarpis_nuo == "2019-01-01"
laikotarpis_condition_21 = balance_21.laikotarpis_nuo == "2020-01-01"


bal_cols = ["obj_kodas", "obj_pav", "form_pav", "nuosavas_kapitalas"]

In [ ]:
top_capital_20 = (
    balance_20[uab_mb_codition_20 & laikotarpis_condition_20][bal_cols]
    .sort_values("nuosavas_kapitalas", ascending=False)
    .dropna()
    .head(100)
)

In [ ]:
top_capital_21.head()

In [ ]:
top_capital_21 = (
    balance_21[uab_mb_codition_21 & laikotarpis_condition_21][bal_cols]
    .sort_values("nuosavas_kapitalas", ascending=False)
    .dropna()
    .head(100)
)

In [ ]:
mb_uab = ["Mažoji bendrija", "Uždaroji akcinė bendrovė"]
pl_uab_mb_codition_20 = pl_20.form_pav.isin(mb_uab)
pl_uab_mb_codition_21 = pl_21.form_pav.isin(mb_uab)

pl_laikotarpis_condition_20 = pl_20.laikotarpis_nuo == "2019-01-01"
pl_laikotarpis_condition_21 = pl_20.laikotarpis_nuo == "2020-01-01"


pl_cols = ["obj_kodas", "obj_pav", "form_pav", "grynasis_pelnas"]

In [ ]:
top_profit_20 = (
    pl_20[pl_uab_mb_codition_20 & pl_laikotarpis_condition_20][pl_cols]
    .sort_values("grynasis_pelnas", ascending=False)
    .dropna()
    .head(100)
)

In [ ]:
top_profit_21 = (
    pl_21[pl_uab_mb_codition_21 & pl_laikotarpis_condition_21][pl_cols]
    .sort_values("grynasis_pelnas", ascending=False)
    .dropna()
    .head(100)
)

In [ ]:
top_profit_21

In [ ]:
best_20 = top_capital_20.merge(top_profit_20, on="obj_kodas")

In [ ]:
best_21 = top_capital_21.merge(top_profit_21, on="obj_kodas")

In [ ]:
best_20

In [ ]:
len(set(top_profit_20.obj_kodas).intersection(set(top_capital_20.obj_kodas)))

In [ ]:
set(top_profit_21.obj_kodas).intersection(set(top_capital_21.obj_kodas))

# 16
Ar topai iš 9 užduoties skiriasi 2020 ir 2021 metais? Kurios įmonės iškrenta iš topų 2021 metais?

In [ ]:
capital_drop_outs_21 = []
for i in top_capital_20.obj_pav:
    if i not in top_capital_21.obj_pav:
        capital_drop_outs_21.append(i)

In [ ]:
capital_drop_outs_21

In [ ]:
profit_drop_outs_21 = []
for i in top_profit_20.obj_pav:
    if i not in top_profit_21.obj_pav:
        profit_drop_outs_21.append(i)

In [ ]:
profit_drop_outs_21